In [1]:
import torch
from torch.utils.data import DataLoader
from model import GPT
from data_fetcher import get_padded_image, get_line, extract_patch, get_line_data
import einops
import types

/home/thetis/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/thetis/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


### Misc Type Printer

In [5]:
def typeTree(thing, indent=0, seen=None, spine="", is_last_child=True):
    if seen is None:
        seen = set()

    current_type = type(thing).__name__

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    prefix = "|__" if indent > 0 else ""
    print(f"{spine}{prefix}{current_type}{dtype_str}")

    if not isinstance(thing, tuple):
        seen.add(current_type)

    new_spine = spine + ("   " if is_last_child else "|  ")

    if isinstance(thing, list):
        for idx, item in enumerate(thing):
            typeTree(
                item, indent + 1, seen, new_spine, is_last_child=(idx == len(thing) - 1)
            )

    elif isinstance(thing, tuple):
        for idx, item in enumerate(thing):
            typeTree(
                item,
                indent + 1,
                set(),
                new_spine,
                is_last_child=(idx == len(thing) - 1),
            )

    elif hasattr(thing, "__getitem__"):
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            typeTree(sample_item, indent + 1, seen, new_spine)

In [3]:
def typeTree(thing, indent=0, seen=None, spine=""):
    if seen is None:
        seen = set()

    current_type = type(thing).__name__

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    prefix = "|__" if indent > 0 else ""
    print(f"{spine.rstrip()}{prefix}{current_type}{dtype_str}")

    if not isinstance(thing, tuple):
        seen.add(current_type)

    # Add a vertical bar to the spine if not the last child, else add space
    new_spine = spine + ("|  " if indent > 0 else "")

    if isinstance(thing, list):
        for idx, item in enumerate(thing):
            is_last_child = idx == len(thing) - 1
            next_spine = new_spine if not is_last_child else spine + "   "
            typeTree(item, indent + 1, seen if not is_last_child else set(), next_spine)

    elif isinstance(thing, tuple):
        for item in thing:
            typeTree(item, indent + 1, set(), new_spine)

    elif hasattr(thing, "__getitem__"):
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            typeTree(sample_item, indent + 1, seen, new_spine)

In [4]:
def typeTree(thing, indent=0, seen=None, spine=""):
    if seen is None:
        seen = set()

    current_type = type(thing).__name__

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    prefix = "|__" if indent > 0 else ""
    print(f"{spine}{prefix}{current_type}{dtype_str}")

    if not isinstance(thing, tuple):
        seen.add(current_type)

    # Add a vertical bar to the spine if not the last child, else add space
    new_spine = spine + ("|  " if indent > 0 else "   ")

    if isinstance(thing, list):
        for idx, item in enumerate(thing):
            is_last_child = idx == len(thing) - 1
            next_spine = new_spine if not is_last_child else spine + "   "
            typeTree(item, indent + 1, seen if not is_last_child else set(), next_spine)

    elif isinstance(thing, tuple):
        for item in thing:
            typeTree(item, indent + 1, set(), new_spine)

    elif hasattr(thing, "__getitem__"):
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            typeTree(sample_item, indent + 1, seen, new_spine)

In [5]:
def typeTree(thing, indent=0, seen=None, spine=""):
    if seen is None:
        seen = set()

    current_type = type(thing).__name__

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    print(f"{spine}|__{current_type}{dtype_str}")

    if not isinstance(thing, tuple):
        seen.add(current_type)

    new_spine = spine + ("|  " if indent > 0 else "   ")

    if isinstance(thing, list):
        for idx, item in enumerate(thing):
            typeTree(
                item, indent + 1, seen if idx < len(thing) - 1 else set(), new_spine
            )

    elif isinstance(thing, tuple):
        for idx, item in enumerate(thing):
            typeTree(item, indent + 1, set(), new_spine)

    elif hasattr(thing, "__getitem__"):
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            typeTree(sample_item, indent + 1, seen, new_spine)

In [6]:
def typeTree(thing, indent=0, seen=None, prefix=""):
    if seen is None:
        seen = set()

    current_type = type(thing).__name__

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen or isinstance(thing, tuple):
        print(f"{prefix}{current_type}{dtype_str}")
        if not isinstance(thing, tuple):
            seen.add(current_type)

    child_prefix = prefix
    if indent > 0:
        child_prefix = prefix[:-2] + "| "

    if isinstance(thing, list):
        local_seen = set()
        for idx, item in enumerate(thing):
            new_prefix = (
                child_prefix + "|__" if idx < len(thing) - 1 else child_prefix + "  |__"
            )
            typeTree(
                item,
                indent + 1,
                seen,
                new_prefix,
            )
            local_seen.add(type(item).__name__)

    elif isinstance(thing, tuple):
        for idx, item in enumerate(thing):
            new_prefix = (
                child_prefix + "|__" if idx < len(thing) - 1 else child_prefix + "  |__"
            )
            typeTree(
                item,
                indent + 1,
                set(),
                new_prefix,
            )

    elif hasattr(thing, "__getitem__"):
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            typeTree(
                sample_item,
                indent + 1,
                seen,
                child_prefix + "  |__",
            )

In [7]:
def typeTree(thing, indent=0, seen=None, is_last_child=True):
    if seen is None:
        seen = set()

    current_type = type(thing).__name__

    # Create the prefix string based on the current level of indentation and whether the node is the last child
    prefix = ("  " * indent) + ("|__" if indent > 0 else "")

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen or isinstance(thing, tuple):
        print(f"{prefix}{current_type}{dtype_str}")
        if not isinstance(thing, tuple):
            seen.add(current_type)

    new_indent = indent + 1

    if isinstance(thing, list):
        local_seen = set()
        for idx, item in enumerate(thing):
            typeTree(
                item,
                new_indent,
                seen,
                is_last_child=idx == len(thing) - 1,
            )
            local_seen.add(type(item).__name__)

    elif isinstance(thing, tuple):
        for idx, item in enumerate(thing):
            typeTree(
                item,
                new_indent,
                set(),
                is_last_child=idx == len(thing) - 1,
            )

    elif hasattr(thing, "__getitem__"):
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            typeTree(
                sample_item,
                new_indent,
                seen,
                is_last_child=True,
            )

In [8]:
def typeTree(thing, indent=0, seen=None, prefix=""):
    if seen is None:
        seen = set()

    current_type = type(thing).__name__

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen or isinstance(thing, tuple):
        print(f"{prefix}{current_type}{dtype_str}")
        if not isinstance(thing, tuple):
            seen.add(current_type)

    if isinstance(thing, list):
        local_seen = set()
        for idx, item in enumerate(thing):
            new_prefix = prefix + ("| " if idx < len(thing) - 1 else "  ")
            typeTree(
                item,
                indent + 2,
                seen,
                new_prefix + "|__",
            )
            local_seen.add(type(item).__name__)

    elif isinstance(thing, tuple):
        for idx, item in enumerate(thing):
            new_prefix = prefix + ("| " if idx < len(thing) - 1 else "  ")
            typeTree(
                item,
                indent + 2,
                set(),
                new_prefix + "|__",
            )

    elif hasattr(thing, "__getitem__"):
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            typeTree(
                sample_item,
                indent + 2,
                seen,
                prefix + "  |__",
            )

In [9]:
def typeTree(thing, indent=0, seen=None, active_branches=None, is_last_child=None):
    if seen is None:
        seen = set()
    if active_branches is None:
        active_branches = []
    if is_last_child is None:
        is_last_child = []

    current_type = type(thing).__name__

    # Construct new active_branches based on is_last_child
    new_active_branches = [not last for last in is_last_child]

    # Create the prefix string using vertical and horizontal bars
    prefix = "".join("| " if active else "  " for active in new_active_branches)
    prefix += "|__" if active_branches else ""

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen or isinstance(thing, tuple):
        print(f"{prefix}{current_type}{dtype_str}")
        if not isinstance(thing, tuple):
            seen.add(current_type)

    if isinstance(thing, list):
        local_seen = set()
        for idx, item in enumerate(thing):
            new_is_last_child = is_last_child + [idx == len(thing) - 1]
            typeTree(
                item,
                indent + 2,
                seen,
                new_active_branches,
                new_is_last_child,
            )
            local_seen.add(type(item).__name__)

    elif isinstance(thing, tuple):
        for idx, item in enumerate(thing):
            new_is_last_child = is_last_child + [idx == len(thing) - 1]
            typeTree(
                item,
                indent + 2,
                set(),
                new_active_branches,
                new_is_last_child,
            )

    elif hasattr(thing, "__getitem__"):
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            new_is_last_child = is_last_child + [True]
            typeTree(
                sample_item,
                indent + 2,
                seen,
                new_active_branches,
                new_is_last_child,
            )

In [10]:
def typeTree(thing, indent=0, seen=None, active_branches=None, is_last_child=None):
    if seen is None:
        seen = set()
    if active_branches is None:
        active_branches = []
    if is_last_child is None:
        is_last_child = []

    current_type = type(thing).__name__

    # Create the prefix string using vertical and horizontal bars
    prefix = "".join(
        "| " if active and not last else "  "
        for active, last in zip(active_branches, is_last_child)
    )
    prefix += "|__" if active_branches else ""

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen or isinstance(thing, tuple):
        print(f"{prefix}{current_type}{dtype_str}")
        if not isinstance(thing, tuple):
            seen.add(current_type)

    if isinstance(thing, list):
        local_seen = set()
        for idx, item in enumerate(thing):
            new_is_last_child = is_last_child + [idx == len(thing) - 1]
            new_active_branches = active_branches + [True]
            if new_is_last_child[-1]:
                new_active_branches[-1] = False
            typeTree(
                item,
                indent + 2,
                seen,
                new_active_branches,
                new_is_last_child,
            )
            local_seen.add(type(item).__name__)

    elif isinstance(thing, tuple):
        for idx, item in enumerate(thing):
            new_is_last_child = is_last_child + [idx == len(thing) - 1]
            new_active_branches = active_branches + [True]
            if new_is_last_child[-1]:
                new_active_branches[-1] = False
            typeTree(
                item,
                indent + 2,
                set(),
                new_active_branches,
                new_is_last_child,
            )

    elif hasattr(thing, "__getitem__"):
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            new_is_last_child = is_last_child + [True]
            new_active_branches = active_branches + [False]
            typeTree(
                sample_item,
                indent + 2,
                seen,
                new_active_branches,
                new_is_last_child,
            )

In [11]:
def typeTree(thing, indent=0, seen=None, active_branches=None, is_last_child=None):
    if seen is None:
        seen = set()
    if active_branches is None:
        active_branches = []
    if is_last_child is None:
        is_last_child = []

    current_type = type(thing).__name__

    # Create the prefix string using vertical and horizontal bars
    prefix = "".join(
        "| " if active and not last else "  "
        for active, last in zip(active_branches, is_last_child)
    )
    prefix += "|__" if active_branches else ""

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen or isinstance(thing, tuple):
        print(f"{prefix}{current_type}{dtype_str}")
        if not isinstance(thing, tuple):
            seen.add(current_type)

    if isinstance(thing, list):
        local_seen = set()
        for idx, item in enumerate(thing):
            item_type = type(item).__name__
            if item_type not in local_seen:
                new_is_last_child = is_last_child + [idx == len(thing) - 1]
                new_active_branches = active_branches + [idx == len(thing) - 1]
                typeTree(
                    item,
                    indent + 2,
                    seen,
                    new_active_branches,
                    new_is_last_child,
                )
                local_seen.add(item_type)

    elif isinstance(thing, tuple):
        for idx, item in enumerate(thing):
            new_is_last_child = is_last_child + [idx == len(thing) - 1]
            new_active_branches = active_branches + [True]
            typeTree(
                item,
                indent + 2,
                set(),  # Reset seen for tuples
                new_active_branches,
                new_is_last_child,
            )

    elif hasattr(thing, "__getitem__"):
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            new_is_last_child = is_last_child + [True]  # Assuming single item for now
            new_active_branches = active_branches + [
                False
            ]  # No more branches to explore
            typeTree(
                sample_item,
                indent + 2,
                seen,
                new_active_branches,
                new_is_last_child,
            )

In [12]:
def typeTree(thing, indent=0, seen=None, active_branches=None, is_last_child=None):
    if seen is None:
        seen = set()
    if active_branches is None:
        active_branches = []
    if is_last_child is None:
        is_last_child = []

    current_type = type(thing).__name__

    # Create the prefix string using vertical and horizontal bars
    prefix = "".join(
        "| " if active and not last else "  "
        for active, last in zip(active_branches, is_last_child)
    )
    prefix += "|__" if active_branches else ""

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen or isinstance(thing, tuple):
        print(f"{prefix}{current_type}{dtype_str}")
        if not isinstance(thing, tuple):
            seen.add(current_type)

    if isinstance(thing, list):
        local_seen = set()
        for idx, item in enumerate(thing):
            item_type = type(item).__name__
            if item_type not in local_seen:
                new_is_last_child = is_last_child + [idx == len(thing) - 1]
                new_active_branches = active_branches + [True]
                typeTree(
                    item,
                    indent + 2,
                    seen,
                    new_active_branches,
                    new_is_last_child,
                )
                local_seen.add(item_type)

    elif isinstance(thing, tuple):
        for idx, item in enumerate(thing):
            new_is_last_child = is_last_child + [idx == len(thing) - 1]
            new_active_branches = active_branches + [True]
            typeTree(
                item,
                indent + 2,
                set(),  # Reset seen for tuples
                new_active_branches,
                new_is_last_child,
            )

    elif hasattr(thing, "__getitem__"):
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            new_is_last_child = is_last_child + [True]  # Assuming single item for now
            new_active_branches = active_branches + [
                False
            ]  # No more branches to explore
            typeTree(
                sample_item,
                indent + 2,
                seen,
                new_active_branches,
                new_is_last_child,
            )

In [13]:
def typeTree(thing, indent=0, seen=None, active_branches=None, is_last_child=None):
    if seen is None:
        seen = set()
    if active_branches is None:
        active_branches = []
    if is_last_child is None:
        is_last_child = []

    current_type = type(thing).__name__

    # Create the prefix string using vertical and horizontal bars
    prefix = "".join(
        "| " if active and not last else "  "
        for active, last in zip(active_branches[:-1], is_last_child[:-1])
    )
    prefix += "|__" if active_branches else ""

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen or isinstance(thing, tuple):
        print(f"{prefix}{current_type}{dtype_str}")
        if not isinstance(thing, tuple):
            seen.add(current_type)

    if isinstance(thing, list):
        local_seen = set()
        for idx, item in enumerate(thing):
            item_type = type(item).__name__
            if item_type not in local_seen:
                new_is_last_child = is_last_child + [idx == len(thing) - 1]
                new_active_branches = active_branches + [True]
                typeTree(item, indent + 2, seen, new_active_branches, new_is_last_child)
                local_seen.add(item_type)

    elif isinstance(thing, tuple):
        for idx, item in enumerate(thing):
            new_is_last_child = is_last_child + [idx == len(thing) - 1]
            new_active_branches = active_branches + [True]
            typeTree(
                item,
                indent + 2,
                set(),  # Reset seen for tuples
                new_active_branches,
                new_is_last_child,
            )

    elif hasattr(thing, "__getitem__"):
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            new_active_branches = active_branches + [True]
            new_is_last_child = is_last_child + [True]  # Assuming single item for now
            typeTree(
                sample_item, indent + 2, seen, new_active_branches, new_is_last_child
            )

In [14]:
def typeTree(thing, indent=0, seen=None, active_branches=None, is_last_child=None):
    if seen is None:
        seen = set()
    if active_branches is None:
        active_branches = []
    if is_last_child is None:
        is_last_child = []

    current_type = type(thing).__name__

    # Create the prefix string using vertical and horizontal bars
    prefix = "".join(
        "| " if active and not last else "  "
        for active, last in zip(active_branches[:-1], is_last_child[:-1])
    )
    prefix += "|__" if active_branches else ""

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen or isinstance(thing, tuple):
        print(f"{prefix}{current_type}{dtype_str}")
        if not isinstance(thing, tuple):
            seen.add(current_type)

    if isinstance(thing, (list, tuple)):
        for idx, item in enumerate(thing):
            new_is_last_child = is_last_child + [idx == len(thing) - 1]
            new_active_branches = active_branches + [True]
            typeTree(
                item,
                indent + 2,
                seen
                if not isinstance(thing, tuple)
                else set(),  # Reset seen for tuples
                new_active_branches,
                new_is_last_child,
            )

    if hasattr(thing, "__getitem__"):
        local_seen = set()
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            item_type = type(sample_item).__name__
            if item_type not in local_seen:
                new_active_branches = active_branches + [True]
                new_is_last_child = is_last_child + [
                    True
                ]  # Assuming single item for now
                typeTree(
                    sample_item,
                    indent + 2,
                    seen,
                    new_active_branches,
                    new_is_last_child,
                )
                local_seen.add(item_type)

In [15]:
def typeTree(thing, indent=0, seen=None, active_branches=None, is_last_child=None):
    if seen is None:
        seen = set()
    if active_branches is None:
        active_branches = []
    if is_last_child is None:
        is_last_child = []

    current_type = type(thing).__name__

    # Create the prefix string using vertical and horizontal bars
    prefix = "".join(
        "| " if active and not last else "  "
        for active, last in zip(active_branches[:-1], is_last_child[:-1])
    )
    prefix += "|__" if active_branches else ""

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen or isinstance(thing, tuple):
        print(f"{prefix}{current_type}{dtype_str}")
        if not isinstance(thing, tuple):
            seen.add(current_type)

    if isinstance(thing, (list, tuple)):
        local_seen = (
            set() if isinstance(thing, tuple) else set()
        )  # Reset local_seen for tuples
        for idx, item in enumerate(thing):
            item_type = type(item).__name__
            if item_type not in local_seen:
                new_is_last_child = is_last_child + [idx == len(thing) - 1]
                new_active_branches = active_branches + [True]
                typeTree(
                    item,
                    indent + 2,
                    seen
                    if not isinstance(thing, tuple)
                    else set(),  # Reset seen for tuples
                    new_active_branches,
                    new_is_last_child,
                )
                local_seen.add(item_type)

    if hasattr(thing, "__getitem__"):
        local_seen = set()
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            item_type = type(sample_item).__name__
            if item_type not in local_seen:
                new_active_branches = active_branches + [True]
                new_is_last_child = is_last_child + [
                    True
                ]  # Assuming single item for now
                typeTree(
                    sample_item,
                    indent + 2,
                    seen,
                    new_active_branches,
                    new_is_last_child,
                )
                local_seen.add(item_type)

In [16]:
def typeTree(thing, indent=0, seen=None, active_branches=None, is_last_child=None):
    if seen is None:
        seen = set()
    if active_branches is None:
        active_branches = []
    if is_last_child is None:
        is_last_child = []

    current_type = type(thing).__name__

    # Create the prefix string using vertical and horizontal bars
    prefix = "".join(
        "| " if active and not last else "  "
        for active, last in zip(active_branches[:-1], is_last_child[:-1])
    )
    prefix += "|__" if active_branches else ""

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen or isinstance(thing, tuple):
        print(f"{prefix}{current_type}{dtype_str}")
        if not isinstance(thing, tuple):
            seen.add(current_type)

    if isinstance(thing, (list, tuple)):
        local_seen = set()
        for idx, item in enumerate(thing):
            item_type = type(item).__name__
            if item_type not in local_seen:
                new_is_last_child = is_last_child + [idx == len(thing) - 1]
                new_active_branches = active_branches + [True]
                typeTree(
                    item,
                    indent + 2,
                    seen
                    if not isinstance(thing, tuple)
                    else set(),  # Reset seen if the current thing is a tuple
                    new_active_branches,
                    new_is_last_child,
                )
                local_seen.add(item_type)

    if hasattr(thing, "__getitem__"):
        local_seen = set()
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            item_type = type(sample_item).__name__
            if item_type not in local_seen:
                new_active_branches = active_branches + [True]
                new_is_last_child = is_last_child + [
                    True
                ]  # Assuming single item for now
                typeTree(
                    sample_item,
                    indent + 2,
                    seen,
                    new_active_branches,
                    new_is_last_child,
                )
                local_seen.add(item_type)

In [17]:
def typeTree(thing, indent=0, seen=None, active_branches=None, is_last_child=None):
    if seen is None:
        seen = set()
    if active_branches is None:
        active_branches = []
    if is_last_child is None:
        is_last_child = []

    current_type = type(thing).__name__

    # Create the prefix string using vertical and horizontal bars
    prefix = "".join(
        "| " if active and not last else "  "
        for active, last in zip(active_branches[:-1], is_last_child[:-1])
    )
    prefix += "|__" if active_branches else ""

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen or isinstance(thing, tuple):
        print(f"{prefix}{current_type}{dtype_str}")
        if current_type != "tuple":
            seen.add(current_type)

    if isinstance(thing, (list, tuple)):
        local_seen = set()
        for idx, item in enumerate(thing):
            item_type = type(item).__name__
            if item_type not in local_seen:
                new_is_last_child = is_last_child + [idx == len(thing) - 1]
                new_active_branches = active_branches + [True]
                typeTree(
                    item,
                    indent + 2,
                    seen if isinstance(thing, tuple) else local_seen,
                    new_active_branches,
                    new_is_last_child,
                )
                local_seen.add(item_type)

    if hasattr(thing, "__getitem__"):
        local_seen = set()
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            item_type = type(sample_item).__name__
            if item_type not in local_seen:
                new_active_branches = active_branches + [True]
                new_is_last_child = is_last_child + [
                    True
                ]  # Assuming single item for now
                typeTree(
                    sample_item,
                    indent + 2,
                    seen,
                    new_active_branches,
                    new_is_last_child,
                )
                local_seen.add(item_type)

In [18]:
def typeTree(thing, indent=0, seen=None, active_branches=None, is_last_child=None):
    if seen is None:
        seen = set()
    if active_branches is None:
        active_branches = []
    if is_last_child is None:
        is_last_child = []

    current_type = type(thing).__name__

    # Create the prefix string using vertical and horizontal bars
    prefix = "".join(
        "| " if active and not last else "  "
        for active, last in zip(active_branches[:-1], is_last_child[:-1])
    )
    prefix += "|__" if active_branches else ""

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen:
        print(f"{prefix}{current_type}{dtype_str}")
        seen.add(current_type)

    if isinstance(thing, (list, tuple)):
        local_seen = set()
        for idx, item in enumerate(thing):
            item_type = type(item).__name__
            if item_type not in local_seen:
                new_is_last_child = is_last_child + [idx == len(thing) - 1]
                new_active_branches = active_branches + [True]
                typeTree(
                    item, indent + 2, local_seen, new_active_branches, new_is_last_child
                )
                local_seen.add(item_type)

    if hasattr(thing, "__getitem__"):
        local_seen = set()
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            item_type = type(sample_item).__name__
            if item_type not in local_seen:
                # Add a new active branch for the child node
                new_active_branches = active_branches + [True]
                new_is_last_child = is_last_child + [
                    True
                ]  # Assuming single item for now
                typeTree(
                    sample_item,
                    indent + 2,
                    local_seen,
                    new_active_branches,
                    new_is_last_child,
                )
                local_seen.add(item_type)

### Type Tree

In [30]:
class TypeTree:
    def __init__(self):
        self.lines = []

    def traverse(self, thing, indent=0, is_last=True, prefix=""):
        # Determine the type of the current object
        current_type = type(thing).__name__
        method_name = f"handle_{current_type.lower()}"
        handler = getattr(self, method_name, self.default_handler)
        handler(thing, indent, is_last, prefix)

    def default_handler(self, thing, indent, is_last, prefix):
        dtype_str = ""
        if hasattr(thing, "dtype"):
            dtype_str = f" (dtype: {thing.dtype})"

        new_prefix = prefix + ("    " if is_last else "|   ")

        # Print the current object type
        self.lines.append(
            f"{prefix}|__{type(thing).__name__}{dtype_str}"
            if indent
            else f"{type(thing).__name__}{dtype_str}"
        )

        if isinstance(thing, str):
            return
        elif hasattr(thing, "__getitem__"):
            items = list(thing)
            for i, item in enumerate(items):
                self.traverse(item, indent + 1, i == len(items) - 1, new_prefix)

    def handle_tensor(self, thing, indent, is_last, prefix):
        dtype_str = f" (dtype: {thing.dtype})"
        new_prefix = prefix + ("    " if is_last else "|   ")

        # Print the current object type
        self.lines.append(
            f"{prefix}|__{type(thing).__name__}{dtype_str}"
            if indent
            else f"{type(thing).__name__}{dtype_str}"
        )

        for i in range(thing.dim()):
            dim_str = f"dim_{i} ({thing.size(i)})"
            self.lines.append(f"{new_prefix}|__{dim_str}")

    def display(self):
        for line in self.lines:
            print(line)

In [21]:
class TypeTree:
    def __init__(self):
        self.lines = []

    def traverse(self, thing, is_last=True, prefix=""):
        current_type = type(thing).__name__
        method_name = f"handle_{current_type.lower()}"
        handler = getattr(self, method_name, self.default_handler)
        handler(thing, is_last, prefix)

    def default_handler(self, thing, is_last, prefix):
        current_type = type(thing).__name__
        dtype_str = f" (dtype: {thing.dtype})" if hasattr(thing, "dtype") else ""
        new_prefix = prefix + ("|   " if not is_last else "    ")
        self.lines.append(f"{prefix}|__{current_type}{dtype_str}")

        if hasattr(thing, "__getitem__") and not isinstance(thing, str):
            items = list(thing)[
                :1
            ]  # Consider only the first item in a homogeneous list
            for i, item in enumerate(items):
                self.traverse(item, i == len(items) - 1, new_prefix)

    def handle_tensor(self, thing, is_last, prefix):
        dtype_str = f" (dtype: {thing.dtype})"
        new_prefix = prefix + ("|   " if not is_last else "    ")
        self.lines.append(f"{prefix}|__Tensor{dtype_str}")

        for i in range(thing.dim()):
            dim_str = f"dim_{i} ({thing.size(i)})"
            self.lines.append(f"{new_prefix}|__{dim_str}")

    def display(self):
        for line in self.lines:
            print(line)

In [3]:
class TypeTree:
    def traverse(
        self, thing, indent=0, seen=None, active_branches=None, is_last_child=None
    ):
        print(
            f"Traverse was called with active_branches: {active_branches}, is_last_child: {is_last_child}"
        )
        if seen is None:
            seen = set()
        if active_branches is None:
            active_branches = []
        if is_last_child is None:
            is_last_child = []

        current_type = type(thing).__name__
        method_name = f"handle_{current_type}"
        handler = getattr(self, method_name, self.default_handler)
        handler(thing, indent, seen, active_branches.copy(), is_last_child.copy())

    def default_handler(self, thing, indent, seen, active_branches, is_last_child):
        current_type = type(thing).__name__
        dtype_str = ""
        if hasattr(thing, "dtype"):
            dtype_str = f" (dtype: {thing.dtype})"

        prefix = "".join(
            "| " if active and not last else "  "
            for active, last in zip(active_branches, is_last_child)
        )
        prefix += "|__" if indent > 0 else ""
        print(f"{prefix}{current_type}{dtype_str}")

        if hasattr(thing, "__getitem__"):
            try:
                sample_item = thing[0]
            except (IndexError, KeyError, TypeError):
                sample_item = None

            if sample_item is not None:
                new_is_last_child = is_last_child + [True]
                new_active_branches = active_branches + [True]
                print(f"Calling self.traverse in default_handler")
                self.traverse(
                    sample_item,
                    indent + 2,
                    seen,
                    new_active_branches,
                    new_is_last_child,
                )

    def handle_list(self, thing, indent, seen, active_branches, is_last_child):
        print(
            f"Handling a list with active_branches: {active_branches}, is_last_child: {is_last_child}"
        )

        self.default_handler(thing, indent, seen, active_branches, is_last_child)
        local_seen = set()
        for idx, item in enumerate(thing):
            is_last = idx == len(thing) - 1
            print(f"Calling self.traverse in handle_list")
            self.traverse(
                item,
                indent + 2,
                local_seen,
                active_branches + [True],
                is_last_child + [is_last],
            )

    def handle_tuple(self, thing, indent, seen, active_branches, is_last_child):
        print(
            f"Handling a tuple with active_branches: {active_branches}, is_last_child: {is_last_child}"
        )

        self.default_handler(thing, indent, seen, active_branches, is_last_child)
        for idx, item in enumerate(thing):
            is_last = idx == len(thing) - 1
            print(f"Calling self.traverse in handle_tuple")
            self.traverse(
                item,
                indent + 2,
                seen,
                active_branches + [True],
                is_last_child + [is_last],
            )

In [20]:
class TypeTree:
    def traverse(
        self, thing, indent=0, seen=None, active_branches=None, is_last_child=None
    ):
        if seen is None:
            seen = set()
        if active_branches is None:
            active_branches = []
        if is_last_child is None:
            is_last_child = []

        current_type = type(thing).__name__
        method_name = f"handle_{current_type}"
        handler = getattr(self, method_name, self.default_handler)
        handler(thing, indent, seen, active_branches.copy(), is_last_child.copy())

    def default_handler(self, thing, indent, seen, active_branches, is_last_child):
        current_type = type(thing).__name__
        dtype_str = ""
        if hasattr(thing, "dtype"):
            dtype_str = f" (dtype: {thing.dtype})"

        prefix = "".join(
            "| " if active and not last else "  "
            for active, last in zip(active_branches, is_last_child)
        )
        prefix += "|__" if active_branches else ""
        print(f"{prefix}{current_type}{dtype_str}")

        if hasattr(thing, "__getitem__"):
            try:
                sample_item = thing[0]
            except (IndexError, KeyError, TypeError):
                sample_item = None

            if sample_item is not None:
                new_is_last_child = is_last_child + [True]
                new_active_branches = active_branches + [True]
                self.traverse(
                    sample_item,
                    indent + 2,
                    seen,
                    new_active_branches,
                    new_is_last_child,
                )

    def handle_list(self, thing, indent, seen, active_branches, is_last_child):
        self.default_handler(thing, indent, seen, active_branches, is_last_child)
        local_seen = set()
        new_is_last_child = is_last_child + [False] * (len(thing) - 1) + [True]
        new_active_branches = active_branches + [True]
        for idx, item in enumerate(thing):
            self.traverse(
                item, indent + 2, local_seen, new_active_branches, new_is_last_child
            )

    def handle_tuple(self, thing, indent, seen, active_branches, is_last_child):
        self.default_handler(thing, indent, seen, active_branches, is_last_child)
        new_is_last_child = is_last_child + [False] * (len(thing) - 1) + [True]
        new_active_branches = active_branches + [True]
        for idx, item in enumerate(thing):
            self.traverse(
                item, indent + 2, seen, new_active_branches, new_is_last_child
            )

In [21]:
class TypeTree:
    def traverse(
        self, thing, indent=0, seen=None, active_branches=None, is_last_child=None
    ):
        if seen is None:
            seen = set()
        if active_branches is None:
            active_branches = []
        if is_last_child is None:
            is_last_child = []

        current_type = type(thing).__name__
        method_name = f"handle_{current_type}"
        handler = getattr(self, method_name, self.default_handler)
        handler(thing, indent, seen, active_branches, is_last_child)

    def default_handler(self, thing, indent, seen, active_branches, is_last_child):
        current_type = type(thing).__name__
        dtype_str = ""
        if hasattr(thing, "dtype"):
            dtype_str = f" (dtype: {thing.dtype})"

        prefix = "".join(
            "| " if active and not last else "  "
            for active, last in zip(active_branches[:-1], is_last_child[:-1])
        )
        prefix += "|__" if active_branches else ""
        print(f"{prefix}{current_type}{dtype_str}")

        if not isinstance(thing, tuple):
            seen.add(current_type)

        if hasattr(thing, "__getitem__"):
            try:
                sample_item = thing[0]
            except (IndexError, KeyError, TypeError):
                sample_item = None

            if sample_item is not None:
                new_is_last_child = is_last_child + [True]
                new_active_branches = active_branches + [True]
                self.traverse(
                    sample_item,
                    indent + 2,
                    seen,
                    new_active_branches,
                    new_is_last_child,
                )

    def handle_list(self, thing, indent, seen, active_branches, is_last_child):
        self.default_handler(thing, indent, seen, active_branches, is_last_child)
        local_seen = set()
        new_is_last_child = is_last_child + [False] * (len(thing) - 1) + [True]
        new_active_branches = active_branches + [True]
        for idx, item in enumerate(thing):
            self.traverse(
                item, indent + 2, local_seen, new_active_branches, new_is_last_child
            )

    def handle_tuple(self, thing, indent, seen, active_branches, is_last_child):
        self.default_handler(thing, indent, seen, active_branches, is_last_child)
        new_is_last_child = is_last_child + [False] * (len(thing) - 1) + [True]
        new_active_branches = active_branches + [True]
        for idx, item in enumerate(thing):
            self.traverse(
                item, indent + 2, seen, new_active_branches, new_is_last_child
            )

In [22]:
class TypeTree:
    def traverse(self, thing, indent=0, seen=None, spine="", is_last_child=True):
        if seen is None:
            seen = set()

        current_type = type(thing).__name__
        method_name = f"handle_{current_type}"
        handler = getattr(self, method_name, self.default_handler)
        handler(thing, indent, seen, spine, is_last_child)

    def default_handler(self, thing, indent, seen, spine, is_last_child):
        current_type = type(thing).__name__
        dtype_str = ""
        if hasattr(thing, "dtype"):
            dtype_str = f" (dtype: {thing.dtype})"

        prefix = "|__" if indent > 0 else ""
        print(f"{spine}{prefix}{current_type}{dtype_str}")

        new_spine = spine + ("   " if is_last_child else "|  ")

        if hasattr(thing, "__getitem__"):
            try:
                sample_item = thing[0]
            except (IndexError, KeyError, TypeError):
                sample_item = None

            if sample_item is not None:
                self.traverse(sample_item, indent + 1, seen, new_spine)

    def handle_list(self, thing, indent, seen, spine, is_last_child):
        self.default_handler(thing, indent, seen, spine, is_last_child)
        new_spine = spine + ("   " if is_last_child else "|  ")
        for idx, item in enumerate(thing):
            self.traverse(
                item, indent + 1, seen, new_spine, is_last_child=(idx == len(thing) - 1)
            )

    def handle_tuple(self, thing, indent, seen, spine, is_last_child):
        self.default_handler(thing, indent, seen, spine, is_last_child)
        new_spine = spine + ("   " if is_last_child else "|  ")
        for idx, item in enumerate(thing):
            self.traverse(
                item, indent + 1, None, new_spine, is_last_child=(idx == len(thing) - 1)
            )

In [23]:
class TypeTree:
    def traverse(self, thing, indent=0, seen=None, spine="", is_last_child=True):
        if seen is None:
            seen = set()

        current_type = type(thing).__name__
        method_name = f"handle_{current_type}"
        handler = getattr(self, method_name, self.default_handler)
        handler(thing, indent, seen, spine, is_last_child)

    def default_handler(self, thing, indent, seen, spine, is_last_child):
        current_type = type(thing).__name__
        dtype_str = ""
        if hasattr(thing, "dtype"):
            dtype_str = f" (dtype: {thing.dtype})"

        prefix = "|__" if indent > 0 else ""
        print(f"{spine}{prefix}{current_type}{dtype_str}")

        if not isinstance(thing, tuple):
            seen.add(current_type)

        new_spine = spine + ("   " if is_last_child else "|  ")

        if hasattr(thing, "__getitem__"):
            try:
                sample_item = thing[0]
            except (IndexError, KeyError, TypeError):
                sample_item = None

            if sample_item is not None:
                self.traverse(sample_item, indent + 1, seen, new_spine)

    def handle_list(self, thing, indent, seen, spine, is_last_child):
        self.default_handler(thing, indent, seen, spine, is_last_child)
        new_spine = spine + ("   " if is_last_child else "|  ")
        for idx, item in enumerate(thing):
            self.traverse(
                item, indent + 1, seen, new_spine, is_last_child=(idx == len(thing) - 1)
            )

    def handle_tuple(self, thing, indent, seen, spine, is_last_child):
        self.default_handler(thing, indent, seen, spine, is_last_child)
        new_spine = spine + ("   " if is_last_child else "|  ")
        for idx, item in enumerate(thing):
            self.traverse(
                item, indent + 1, seen, new_spine, is_last_child=(idx == len(thing) - 1)
            )

In [24]:
class TypeTree:
    def traverse(self, thing, indent=0, seen=None, spine="", is_last_child=True):
        if seen is None:
            seen = set()

        current_type = type(thing).__name__
        method_name = f"handle_{current_type}"
        handler = getattr(self, method_name, self.default_handler)
        handler(thing, indent, seen, spine, is_last_child)

    def default_handler(self, thing, indent, seen, spine, is_last_child):
        current_type = type(thing).__name__
        dtype_str = ""
        if hasattr(thing, "dtype"):
            dtype_str = f" (dtype: {thing.dtype})"

        prefix = "|__" if indent > 0 else ""
        print(f"{spine}{prefix}{current_type}{dtype_str}")

        if not isinstance(thing, tuple):
            seen.add(current_type)

        new_spine = spine + ("   " if is_last_child else "|  ")
        if hasattr(thing, "__getitem__"):
            try:
                sample_item = thing[0]
            except (IndexError, KeyError, TypeError):
                sample_item = None

            if sample_item is not None:
                self.traverse(sample_item, indent + 1, seen, new_spine)

    def handle_list(self, thing, indent, seen, spine, is_last_child):
        self.default_handler(thing, indent, seen, spine, is_last_child)
        new_spine = spine + ("   " if is_last_child else "|  ")
        for idx, item in enumerate(thing):
            self.traverse(
                item, indent + 1, seen, new_spine, is_last_child=(idx == len(thing) - 1)
            )

    def handle_tuple(self, thing, indent, seen, spine, is_last_child):
        self.default_handler(thing, indent, seen, spine, is_last_child)
        new_spine = spine + ("   " if is_last_child else "|  ")
        for idx, item in enumerate(thing):
            self.traverse(
                item,
                indent + 1,
                set(),
                new_spine,
                is_last_child=(idx == len(thing) - 1),
            )

In [25]:
class TypeTree:
    def traverse(self, thing, indent=0, seen=None, spine="", is_last_child=True):
        if seen is None:
            seen = set()

        current_type = type(thing).__name__
        method_name = f"handle_{current_type}"
        handler = getattr(self, method_name, self.default_handler)
        handler(thing, indent, seen, spine, is_last_child)

    def default_handler(self, thing, indent, seen, spine, is_last_child):
        current_type = type(thing).__name__
        dtype_str = ""
        if hasattr(thing, "dtype"):
            dtype_str = f" (dtype: {thing.dtype})"

        prefix = "|__" if indent > 0 else ""
        print(f"{spine}{prefix}{current_type}{dtype_str}")

        if not isinstance(thing, tuple):
            seen.add(current_type)

        new_spine = spine + ("   " if is_last_child else "|  ")

        if hasattr(thing, "__getitem__"):
            try:
                sample_item = thing[0]
            except (IndexError, KeyError, TypeError):
                sample_item = None

            if sample_item is not None:
                self.traverse(sample_item, indent + 1, seen, new_spine)

    def handle_list(self, thing, indent, seen, spine, is_last_child):
        self.default_handler(thing, indent, seen, spine, is_last_child)
        for idx, item in enumerate(thing):
            self.traverse(
                item,
                indent + 1,
                seen,
                spine + "|  ",
                is_last_child=(idx == len(thing) - 1),
            )

    def handle_tuple(self, thing, indent, seen, spine, is_last_child):
        self.default_handler(thing, indent, seen, spine, is_last_child)
        for idx, item in enumerate(thing):
            self.traverse(
                item,
                indent + 1,
                set(),
                spine + "|  ",
                is_last_child=(idx == len(thing) - 1),
            )

In [26]:
class TypeTree:
    def __init__(self):
        self.indent = 0
        self.seen = set()
        self.spine = ""
        self.is_last_child = True

    def traverse(self, thing, indent=0, seen=None, spine="", is_last_child=True):
        if seen is None:
            seen = set()

        self.indent = indent
        self.seen = seen
        self.spine = spine
        self.is_last_child = is_last_child

        current_type = type(thing).__name__
        method_name = f"handle_{current_type}"
        handler = getattr(self, method_name, self.default_handler)
        handler(thing)

    def default_handler(self, thing):
        dtype_str = ""
        if hasattr(thing, "dtype"):
            dtype_str = f" (dtype: {thing.dtype})"

        prefix = "|__" if self.indent > 0 else ""
        print(f"{self.spine}{prefix}{type(thing).__name__}{dtype_str}")

        if not isinstance(thing, tuple):
            self.seen.add(type(thing).__name__)

        new_spine = self.spine + ("   " if self.is_last_child else "|  ")
        if hasattr(thing, "__getitem__"):
            try:
                sample_item = thing[0]
            except (IndexError, KeyError, TypeError):
                sample_item = None
            if sample_item is not None:
                self.traverse(sample_item, self.indent + 1, self.seen, new_spine)

    def handle_list(self, thing):
        self.default_handler(thing)
        for idx, item in enumerate(thing):
            self.traverse(
                item,
                self.indent + 1,
                self.seen,
                self.spine + "|  ",
                is_last_child=(idx == len(thing) - 1),
            )

    def handle_tuple(self, thing):
        self.default_handler(thing)
        for idx, item in enumerate(thing):
            self.traverse(
                item,
                self.indent + 1,
                set(),
                self.spine + "|  ",
                is_last_child=(idx == len(thing) - 1),
            )

    # Add methods for any other types you wish to handle specially.
    # For example:
    # def handle_Subset(self, thing):
    #     ...

### Get Data

In [27]:
image = get_padded_image("./juneau2k.jpg")

/opt/miniconda3/lib/python3.10/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (122880000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [28]:
first_line = get_line(0)

In [29]:
first_line.shape

torch.Size([1372, 2])

In [30]:
jumps = (first_line[1:] - first_line[:-1]).round()

In [31]:
jumps

tensor([[6., -0.],
        [7., 0.],
        [7., 0.],
        ...,
        [2., 0.],
        [3., 1.],
        [2., -0.]], dtype=torch.float64)

In [32]:
"""
jumps tells us the diff between point n+1 and n. Starting at 0. 
Since we're utilizing the first 2 data points, we use the 0th and 1st points, and our target is the 2nd - 1st points. 
"""
input_points = list(zip(first_line[:-2], first_line[:-1]))

In [33]:
jumps = (first_line[1:] - first_line[:-1]).round()

In [34]:
target_points = jumps[1:]

In [35]:
type(input_points[0])

tuple

In [36]:
line = get_line(0)

In [37]:
line.shape

torch.Size([1372, 2])

In [ ]:
train_data, test_data = get_line_data(0)

In [6]:
torch_types = {
    name: cls for name, cls in torch.__dict__.items() if isinstance(cls, type)
}

typeTree(get_line_data(0))

list
   |__Subset
   |  |__tuple
   |     |__Tensor (dtype: torch.float64)
   |     |  |__Tensor (dtype: torch.float64)
   |     |     |__Tensor (dtype: torch.float64)
   |     |__Tensor (dtype: torch.float64)
   |        |__Tensor (dtype: torch.float64)
   |__Subset
      |__tuple
         |__Tensor (dtype: torch.float64)
         |  |__Tensor (dtype: torch.float64)
         |     |__Tensor (dtype: torch.float64)
         |__Tensor (dtype: torch.float64)
            |__Tensor (dtype: torch.float64)


In [7]:
"""
list of subsets of tuples of tensors
but what I want is a list of 
"""
d = get_line_data(0)
typeTree(d)

list
   |__Subset
   |  |__tuple
   |     |__Tensor (dtype: torch.float64)
   |     |  |__Tensor (dtype: torch.float64)
   |     |     |__Tensor (dtype: torch.float64)
   |     |__Tensor (dtype: torch.float64)
   |        |__Tensor (dtype: torch.float64)
   |__Subset
      |__tuple
         |__Tensor (dtype: torch.float64)
         |  |__Tensor (dtype: torch.float64)
         |     |__Tensor (dtype: torch.float64)
         |__Tensor (dtype: torch.float64)
            |__Tensor (dtype: torch.float64)


In [8]:
line = get_line(0)

jumps = (line[1:] - line[:-1]).round()
output = list(zip(torch.stack([line[:-2], line[1:-1]]), jumps[1:]))

In [ ]:
line = torch.randn(10, 2)
jumps = (line[1:] - line[:-1]).round()
s = torch.stack([line[:-2], line[1:-1]], dim=1)

In [ ]:
output = list(zip(s, jumps[1:]))

In [ ]:
s

tensor([[[ 0.9962, -0.5673],
         [ 1.5197,  0.8453]],

        [[ 1.5197,  0.8453],
         [ 0.6859, -0.0692]],

        [[ 0.6859, -0.0692],
         [-1.2299, -1.0222]],

        [[-1.2299, -1.0222],
         [ 0.0463,  1.3502]],

        [[ 0.0463,  1.3502],
         [-0.1309, -1.5546]],

        [[-0.1309, -1.5546],
         [-1.9320, -0.5457]],

        [[-1.9320, -0.5457],
         [ 1.7011, -0.4821]],

        [[ 1.7011, -0.4821],
         [-0.4964, -2.1380]]])

In [ ]:
output[0]

(tensor([[ 0.9962, -0.5673],
         [ 1.5197,  0.8453]]),
 tensor([-1., -1.]))

In [9]:
line = torch.randn(10, 2)
jumps = (line[1:] - line[:-1]).round()
output = list(zip(torch.stack([line[:-2], line[1:-1]], dim=1), jumps[1:]))
typeTree(output)

list
   |__tuple
   |  |__Tensor (dtype: torch.float32)
   |  |  |__Tensor (dtype: torch.float32)
   |  |     |__Tensor (dtype: torch.float32)
   |  |__Tensor (dtype: torch.float32)
   |     |__Tensor (dtype: torch.float32)
   |__tuple
   |  |__Tensor (dtype: torch.float32)
   |  |  |__Tensor (dtype: torch.float32)
   |  |     |__Tensor (dtype: torch.float32)
   |  |__Tensor (dtype: torch.float32)
   |     |__Tensor (dtype: torch.float32)
   |__tuple
   |  |__Tensor (dtype: torch.float32)
   |  |  |__Tensor (dtype: torch.float32)
   |  |     |__Tensor (dtype: torch.float32)
   |  |__Tensor (dtype: torch.float32)
   |     |__Tensor (dtype: torch.float32)
   |__tuple
   |  |__Tensor (dtype: torch.float32)
   |  |  |__Tensor (dtype: torch.float32)
   |  |     |__Tensor (dtype: torch.float32)
   |  |__Tensor (dtype: torch.float32)
   |     |__Tensor (dtype: torch.float32)
   |__tuple
   |  |__Tensor (dtype: torch.float32)
   |  |  |__Tensor (dtype: torch.float32)
   |  |     |__Tensor (dty

In [ ]:
output[0]

(tensor([[ 1.4849, -0.5678],
         [-0.1166,  0.9049]]),
 tensor([ 1., -1.]))

In [ ]:
list(zip(jumps[1:], jumps[1:]))

[(tensor([ 1., -1.]), tensor([ 1., -1.])),
 (tensor([-2.,  1.]), tensor([-2.,  1.])),
 (tensor([1., 1.]), tensor([1., 1.])),
 (tensor([ 2., -3.]), tensor([ 2., -3.])),
 (tensor([-1.,  1.]), tensor([-1.,  1.])),
 (tensor([-1.,  2.]), tensor([-1.,  2.])),
 (tensor([-2., -1.]), tensor([-2., -1.])),
 (tensor([2., 0.]), tensor([2., 0.]))]

In [ ]:
output[0]

(tensor([[ 1.4849, -0.5678],
         [-0.1166,  0.9049]]),
 tensor([ 1., -1.]))

In [11]:
a = torch.randn(2, 3)
b = torch.randn(2)

In [22]:
t = TypeTree()

In [23]:
thing = (((1, 2), (1, 2)), (1, 2))
typeTree((((1, 2), (1, 2)), (1, 2)))
t.traverse(thing)

tuple
   |__tuple
   |  |__tuple
   |  |  |__int
   |  |  |__int
   |  |__tuple
   |     |__int
   |     |__int
   |__tuple
      |__int
      |__int


In [24]:
typeTree(a)
t.traverse(a)

Tensor (dtype: torch.float32)
   |__Tensor (dtype: torch.float32)
      |__Tensor (dtype: torch.float32)


In [15]:
c = torch.tensor(1)

In [29]:
typeTree((a, b, c))
t = TypeTree()
t.traverse((a, b, c))
t.display()

tuple
   |__Tensor (dtype: torch.float32)
   |  |__Tensor (dtype: torch.float32)
   |     |__Tensor (dtype: torch.float32)
   |__Tensor (dtype: torch.float32)
   |  |__Tensor (dtype: torch.float32)
   |__Tensor (dtype: torch.int64)
|__tuple
    |__Tensor (dtype: torch.float32)
        |__dim_0 (2)
        |__dim_1 (3)


In [20]:
typeTree([([(1, 2)], 1)])

list
   |__tuple
      |__list
      |  |__tuple
      |     |__int
      |     |__int
      |__int


In [ ]:
typeTree(output)

list
|__tuple
| |__Tensor (dtype: torch.float32)
| | |__Tensor (dtype: torch.float32)
| |   |__Tensor (dtype: torch.float32)
| |__Tensor (dtype: torch.float32)
|   |__Tensor (dtype: torch.float32)
|     |__Tensor (dtype: torch.float32)
|__tuple
  |__Tensor (dtype: torch.float32)
  | |__Tensor (dtype: torch.float32)
  |   |__Tensor (dtype: torch.float32)
  |__Tensor (dtype: torch.float32)
    |__Tensor (dtype: torch.float32)
      |__Tensor (dtype: torch.float32)


In [ ]:
a = [[1, 1], [2, 2], [3, 3]]
b = [[4, 4], [5, 5], [6, 6]]
c = [[7, 7], [8, 8], [9, 9]]

In [ ]:
list(zip(a, b))

[([1, 1], [4, 4]), ([2, 2], [5, 5]), ([3, 3], [6, 6])]

In [ ]:
output[0]

(tensor([[ 1.4849, -0.5678],
         [-0.1166,  0.9049]]),
 tensor([ 1., -1.]))

In [ ]:
typeTree(line)
typeTree(jumps)
typeTree(output)

Tensor (dtype: torch.float32)
|__Tensor (dtype: torch.float32)
  |__Tensor (dtype: torch.float32)
Tensor (dtype: torch.float32)
|__Tensor (dtype: torch.float32)
  |__Tensor (dtype: torch.float32)
list
|__tuple
| |__Tensor (dtype: torch.float32)
| | |__Tensor (dtype: torch.float32)
| |   |__Tensor (dtype: torch.float32)
| |__Tensor (dtype: torch.float32)
|   |__Tensor (dtype: torch.float32)
|     |__Tensor (dtype: torch.float32)
|__tuple
  |__Tensor (dtype: torch.float32)
  | |__Tensor (dtype: torch.float32)
  |   |__Tensor (dtype: torch.float32)
  |__Tensor (dtype: torch.float32)
    |__Tensor (dtype: torch.float32)
      |__Tensor (dtype: torch.float32)


In [ ]:
len(d[0][0])

2

### Get Model

In [44]:
model = GPT(num_layers=3, d_model=128, num_heads=8, d_hidden=512, input_dim=912)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = torch.nn.functional.mse_loss

### Training

In [45]:
image = get_padded_image("./juneau2k.jpg")
train_data, test_data = get_line_data(0)
print(list(test_data))
print(typeTree(train_data))

/opt/miniconda3/lib/python3.10/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (122880000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[(tensor([[5715.5925, 2466.7177],
        [5719.2166, 2464.7662]], dtype=torch.float64), tensor([ 2., -3.], dtype=torch.float64)), (tensor([[7281.2525, 3186.4082],
        [7285.3080, 3188.1287]], dtype=torch.float64), tensor([4., 2.], dtype=torch.float64)), (tensor([[4531.2521, 2481.4929],
        [4530.0441, 2479.1697]], dtype=torch.float64), tensor([-2., -3.], dtype=torch.float64)), (tensor([[8046.6384, 3692.7818],
        [8049.9837, 3692.7818]], dtype=torch.float64), tensor([4., 0.], dtype=torch.float64)), (tensor([[7722.0750, 3468.9426],
        [7724.1642, 3470.2945]], dtype=torch.float64), tensor([2., 1.], dtype=torch.float64)), (tensor([[6716.9271, 2985.1439],
        [6722.7216, 2985.7092]], dtype=torch.float64), tensor([5., -0.], dtype=torch.float64)), (tensor([[4827.4069, 2440.3267],
        [4831.0310, 2438.8399]], dtype=torch.float64), tensor([ 4., -1.], dtype=torch.float64)), (tensor([[4571.3961, 2479.0768],
        [4574.7414, 2479.4485]], dtype=torch.float64), tensor([

In [46]:
d = get_line_data(0)
train, test = d[0], d[1]

In [47]:
typeTree(d)

list
|__Subset
| |__tuple
|   |__Tensor (dtype: torch.float64)
|   | |__Tensor (dtype: torch.float64)
|   |   |__Tensor (dtype: torch.float64)
|   |__Tensor (dtype: torch.float64)
|     |__Tensor (dtype: torch.float64)
|       |__Tensor (dtype: torch.float64)
|__Subset
  |__tuple
    |__Tensor (dtype: torch.float64)
    | |__Tensor (dtype: torch.float64)
    |   |__Tensor (dtype: torch.float64)
    |__Tensor (dtype: torch.float64)
      |__Tensor (dtype: torch.float64)
        |__Tensor (dtype: torch.float64)


In [48]:
type(list(train))

list

In [49]:
size = 9
for batch_idx, (data, target) in enumerate(list(train)[:10]):
    optimizer.zero_grad()
    previous_location = extract_patch(image, data[0], size)
    current_location = extract_patch(image, data[1], size)
    input = torch.stack([previous_location, current_location])
    prediction = model(input)
    loss = loss_fn(prediction, target)
    loss.backward()
    optimizer.step()

AttributeError: 'Embedding' object has no attribute 'size'

### Misc, recursive type displayer

In [ ]:
def typeTree(thing, indent=0, seen=None, active_branches=None, is_last_child=None):
    if seen is None:
        seen = set()
    if active_branches is None:
        active_branches = []
    if is_last_child is None:
        is_last_child = []

    current_type = type(thing).__name__

    # Create the prefix string using vertical and horizontal bars
    prefix = "".join(
        "| " if active and not last else "  "
        for active, last in zip(active_branches[:-1], is_last_child[:-1])
    )
    prefix += "|__" if active_branches else ""

    dtype_str = ""
    if hasattr(thing, "dtype"):
        dtype_str = f" (dtype: {thing.dtype})"

    if current_type not in seen:
        print(f"{prefix}{current_type}{dtype_str}")
        seen.add(current_type)

    if isinstance(thing, (list, tuple)):
        local_seen = set()
        for idx, item in enumerate(thing):
            item_type = type(item).__name__
            if item_type not in local_seen:
                new_is_last_child = is_last_child + [idx == len(thing) - 1]
                new_active_branches = active_branches + [True]
                typeTree(
                    item, indent + 2, local_seen, new_active_branches, new_is_last_child
                )
                local_seen.add(item_type)

    if hasattr(thing, "__getitem__"):
        local_seen = set()
        try:
            sample_item = thing[0]
        except (IndexError, KeyError, TypeError):
            sample_item = None

        if sample_item is not None:
            item_type = type(sample_item).__name__
            if item_type not in local_seen:
                # Add a new active branch for the child node
                new_active_branches = active_branches + [True]
                new_is_last_child = is_last_child + [
                    True
                ]  # Assuming single item for now
                typeTree(
                    sample_item,
                    indent + 2,
                    local_seen,
                    new_active_branches,
                    new_is_last_child,
                )
                local_seen.add(item_type)